## **ngrok을 사용하여 dagster를 Run !**

## **Step 1: 설치 및 설정**
> ### ngrok에서 인증키 입력


In [ ]:
# Install necessary packages
!pip install dagster dagit pyngrok requests pandas

# Import ngrok and set up the authentication token
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok auth token
ngrok.set_auth_token('???')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## **Step 2: Dagster 파이프라인 코드 작성**

In [ ]:
# Create a new file to define your Dagster pipeline
pipeline_code = """
import json
import requests
import pandas as pd
from dagster import job, op, repository

@op
def load_topstory_ids():
    with open("topstory_ids.json", "r") as f:
        topstory_ids = json.load(f)
    return topstory_ids

@op
def fetch_topstories(topstory_ids):
    results = []
    for item_id in topstory_ids:
        item = requests.get(f"https://hacker-news.firebaseio.com/v0/item/{item_id}.json").json()
        results.append(item)
        if len(results) % 20 == 0:
            print(f"Got {len(results)} items so far.")
    return results

@op
def save_topstories(results):
    df = pd.DataFrame(results)
    df.to_csv("topstories.csv", index=False)

@job
def topstories_job():
    topstory_ids = load_topstory_ids()
    topstories = fetch_topstories(topstory_ids)
    save_topstories(topstories)

@repository
def my_repository():
    return [topstories_job]
"""

In [ ]:
import json

with open("dagster_pipeline.py", "w") as f:
    f.write(pipeline_code)

# Create the topstory_ids.json file
topstory_ids = [8863, 2921983, 121003, 192327, 126809, 102351]  # Example IDs
with open("topstory_ids.json", "w") as f:
    json.dump(topstory_ids, f)


## **Step 3: ngrok 터널 설정 및 Dagit 실행**

In [ ]:
import subprocess
from pyngrok import ngrok

# Run Dagit in the background
dagit_proc = subprocess.Popen(["dagit", "-f", "dagster_pipeline.py", "-h", "0.0.0.0", "-p", "3000"])

# Start ngrok tunnel
public_url = ngrok.connect(3000)
print(f"Dagit UI is available at {public_url}")

# Allow some time for the user to use the Dagit UI
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping ngrok and Dagit...")

# Stop Dagit and ngrok
dagit_proc.terminate()
ngrok.kill()


Dagit UI is available at NgrokTunnel: "https://4b75-34-66-63-190.ngrok-free.app" -> "http://localhost:3000"
Stopping ngrok and Dagit...


### **실습**

In [ ]:
## json 파일
menu = {
    "breakfast": {
        "hours": "7-11",
        "items": {
            "breakfast burritos": "$6.00",
            "pancakes": "$4.00"
        }
    },
    "lunch": {
        "hours": "11-3",
        "items": {
            "hamburger": "$5.00"
        }
    },
    "dinner": {
        "hours": "3-10",
        "items": {
            "spaghetti": "$8.00"
        }
    }
}


# python dictionary is converted to JSON-formatted string
menu_json = json.dumps(menu)
# output the JSON string to the console
print(menu_json)


{"breakfast": {"hours": "7-11", "items": {"breakfast burritos": "$6.00", "pancakes": "$4.00"}}, "lunch": {"hours": "11-3", "items": {"hamburger": "$5.00"}}, "dinner": {"hours": "3-10", "items": {"spaghetti": "$8.00"}}}


In [ ]:

# takes the JSON string menu_json and parses it back into a dictionary.
menu2 = json.loads(menu_json)
print(menu2)


{'breakfast': {'hours': '7-11', 'items': {'breakfast burritos': '$6.00', 'pancakes': '$4.00'}}, 'lunch': {'hours': '11-3', 'items': {'hamburger': '$5.00'}}, 'dinner': {'hours': '3-10', 'items': {'spaghetti': '$8.00'}}}


In [ ]:
# JSON 파일에 쓰기
import os

os.makedirs('data', exist_ok=True)

with open('./data/topstory_ids.json', 'w') as json_file:
    json.dump(menu, json_file)

In [ ]:
import json
import requests
import pandas as pd
from dagster import job, op

@op
def load_topstory_ids():
    with open("./data/topstory_ids.json", "r") as f:
        topstory_ids = json.load(f)
    return topstory_ids

@op
def fetch_topstories(topstory_ids):
    results = []
    for item_id in topstory_ids:
        item = requests.get(f"https://hacker-news.firebaseio.com/v0/item/{item_id}.json").json()
        results.append(item)
        if len(results) % 20 == 0:
            print(f"Got {len(results)} items so far.")
    return results

@op
def save_topstories(results):
    df = pd.DataFrame(results)
    df.to_csv("topstories.csv", index=False)

@job
def topstories_job():
    topstory_ids = load_topstory_ids()
    topstories = fetch_topstories(topstory_ids)
    save_topstories(topstories)

# 실행
if __name__ == "__main__":
    result = topstories_job.execute_in_process()


2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - RUN_START - Started execution of run for "topstories_job".
2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - ENGINE_EVENT - Executing steps in process (pid: 348)
2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - RESOURCE_INIT_STARTED - Starting initialization of resources [io_manager].
2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - RESOURCE_INIT_SUCCESS - Finished initialization of resources [io_manager].
2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - LOGS_CAPTURED - Started capturing logs in process (pid: 348).
2024-05-19 02:01:06 +0000 - dagster - DEBUG - topstories_job - 92b9e0ba-8eaa-49ed-adfe-fd6457424952 - 348 - load_topstory_ids - STEP_STA

In [ ]:
%%writefile dagster_pipeline.py

import json
import requests
import pandas as pd
from dagster import job, op

@op
def load_topstory_ids():
    with open("./data/topstory_ids.json", "r") as f:
        topstory_ids = json.load(f)
    return topstory_ids

@op
def fetch_topstories(topstory_ids):
    results = []
    for item_id in topstory_ids:
        item = requests.get(f"https://hacker-news.firebaseio.com/v0/item/{item_id}.json").json()
        results.append(item)
        if len(results) % 20 == 0:
            print(f"Got {len(results)} items so far.")
    return results

@op
def save_topstories(results):
    df = pd.DataFrame(results)
    df.to_csv("topstories.csv", index=False)

@job
def topstories_job():
    topstory_ids = load_topstory_ids()
    topstories = fetch_topstories(topstory_ids)
    save_topstories(topstories)

# 실행
if __name__ == "__main__":
    result = topstories_job.execute_in_process()


Writing dagster_pipeline.py


In [ ]:
# Run Dagit
!dagit -f dagster_pipeline.py -h 0.0.0.0 -p 3000

2024-05-19 02:07:21 +0000 - dagit - WARNING - The `dagit` CLI command is deprecated and will be removed in dagster 2.0. Please use `dagster-webserver` instead.
2024-05-19 02:07:21 +0000 - dagit - INFO - Using temporary directory /content/tmpdl85pc2c for storage. This will be removed when dagster-webserver exits.
2024-05-19 02:07:21 +0000 - dagit - INFO - To persist information across sessions, set the environment variable DAGSTER_HOME to a directory to use.
2024-05-19 02:07:23 +0000 - dagster.code_server - INFO - Starting Dagster code server for file dagster_pipeline.py in process 10545
2024-05-19 02:07:24 +0000 - dagster.code_server - INFO - Started Dagster code server for file dagster_pipeline.py in process 10545

  Telemetry:

  As an open-source project, we collect usage statistics to inform development priorities. For more
  information, read https://docs.dagster.io/getting-started/telemetry.

  We will not see or store any data that is processed by your code.

  To opt-out, add t